In [1]:
from code.data_processing import *
from code.apiwrappers import BinanceWrapper
from code.charting import chart_to_image, chart_to_arr
from PIL import Image
from code.datasets import DFDataset, ChartImageDataset
from torch.utils.data import *

import torch

import pandas as pd

In [5]:
bw = BinanceWrapper('5lJ0uGit9PuUxHka3hBWhPmsi7dWyxEwvEntUZFKmm0xfNz3VjHWi5WSr5W1VBJV',
                                                      'BFWVs8ko7Cd4sjdQ9amGJTnToGWy9TbQWIjeorSCj23FGiwFaknzkgLPcrgWrxsw')

In [ ]:
# Fetch stats from Binance
candles = bw.historical_candles(start_time='January 1 2019', end_time='June 1 2019', symbol='ETHBTC', interval='1m')

In [7]:
# We'll save the candles to csv as requesting it from Binance takes a lot of time
candles.to_csv('bitcoin1m-GOOD.csv', index=False)

In [4]:
# Read from saved csv instead of fetching it all the time
candles = pd.read_csv('bitcoin1m-GOOD.csv', engine='python')[150000:]

In [5]:
print(candles.count)

<bound method DataFrame.count of                 time      open     close      high       low   volume
150000  1.555322e+12  0.032415  0.032401  0.032416  0.032394   77.326
150001  1.555322e+12  0.032408  0.032400  0.032408  0.032384   58.623
150002  1.555323e+12  0.032399  0.032399  0.032408  0.032393   32.676
150003  1.555323e+12  0.032399  0.032402  0.032420  0.032397   46.297
150004  1.555323e+12  0.032408  0.032404  0.032412  0.032399   45.754
...              ...       ...       ...       ...       ...      ...
216476  1.559347e+12  0.031249  0.031297  0.031298  0.031241  135.250
216477  1.559347e+12  0.031288  0.031286  0.031291  0.031266   99.446
216478  1.559347e+12  0.031280  0.031332  0.031339  0.031280  172.482
216479  1.559347e+12  0.031338  0.031314  0.031343  0.031301   96.830
216480  1.559347e+12  0.031314  0.031357  0.031366  0.031290  547.397

[66481 rows x 6 columns]>


In [13]:
candles = candles[30000:]

In [15]:
candles = candles.reset_index(drop=True)

In [8]:
candles.head()

,time,open,close,high,low,volume,sma20,macd,obv,bb20_low,bb20_mid,bb20_up
150024,1.555324e+12,0.032359,0.032357,0.032374,0.032357,40.877,0.032397,0.128058,-1016.156,0.032359,0.032397,0.032435
150025,1.555324e+12,0.032363,0.032331,0.032363,0.032317,109.847,0.032395,0.103772,-1126.003,0.032353,0.032395,0.032436
150026,1.555324e+12,0.032331,0.032311,0.032331,0.032295,175.491,0.032391,0.081202,-1301.494,0.032342,0.032391,0.032439
150027,1.555324e+12,0.032312,0.032327,0.032330,0.032299,164.230,0.032385,0.060246,-1137.264,0.032327,0.032385,0.032443
150028,1.555324e+12,0.032327,0.032336,0.032349,0.032318,52.082,0.032381,0.040804,-1085.182,0.032319,0.032381,0.032442


In [7]:
candles = add_ti(candles)

In [10]:
inputs, labels = candles_to_inputs_and_labels(candles, num_rows=30, step=10, return_period=5)

In [ ]:
s = int(len(inputs) * 0.7)
train_ds = ChartImageDataset(inputs[:s], labels[:s])
valid_ds = ChartImageDataset(inputs[s:], labels[s:])

In [ ]:
train_dl = DataLoader(train_ds, drop_last=True, **params)
valid_dl = DataLoader(valid_ds, drop_last=True, **params)

In [ ]:
optim = torch.optim.Adam(model.parameters(), lr)